In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x
def H4(n,m): #not as published!
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return (S(base6(n,m)^(1/(m-2))).polynomial())
def deltaStar(n,m):
    jay=j(n+1,m)
    djay=x*derivative(jay,x)
    numerator=djay^(2*m)
    denominator=jay^(2*m-2)*(jay-2^6*m^3)^m
    base=numerator/denominator
    xp=1/(m-2)
    power=S(base)^xp
    return (power.polynomial()).truncate(n+1)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def deltaDiamond(n,m):
    return D(S(expand((H4(n+2,m)^3)/j(n+2,m)))).truncate(n+1)
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)
def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))/1728
def first(list):return list[0]
def last(list):return list[-1]
print("ok1")

ok1


In [9]:
for m in [3..12]:
    print "-----------------------------------------------------------------------"
    print deltaDagger(5,m)
    

-----------------------------------------------------------------------
4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x
-----------------------------------------------------------------------
-11033903104/9*x^5 - 466313216/27*x^4 - 894976/9*x^3 - 15200/27*x^2 + 58/27*x
-----------------------------------------------------------------------
-5193731694125/81*x^5 - 52964129375/162*x^4 - 66973250/81*x^3 - 159125/54*x^2 + 215/54*x
-----------------------------------------------------------------------
-1176821039104*x^5 - 9269411840/3*x^4 - 11468800/3*x^3 - 10240*x^2 + 20/3*x
-----------------------------------------------------------------------
-1671287131816288/135*x^5 - 179312406385/9*x^4 - 12605936*x^3 - 763763/27*x^2 + 280/27*x
-----------------------------------------------------------------------
-90386789826560*x^5 - 8071459766272/81*x^4 - 2635644928/81*x^3 - 1810048/27*x^2 + 412/27*x
-----------------------------------------------------------------------
-506599355441907*x^5 - 8276180

In [7]:
deltaDagger(4,3)

-1472*x^4 + 252*x^3 - 24*x^2 + x

In [8]:
deltaDagger(4,4)

-466313216/27*x^4 - 894976/9*x^3 - 15200/27*x^2 + 58/27*x

In [9]:
deltaDagger(4,5)

-52964129375/162*x^4 - 66973250/81*x^3 - 159125/54*x^2 + 215/54*x

In [10]:
poly=D(deltaDagger(100,3))
data=[]
ls=poly.list()
for n in [1..len(ls)-1]:
    data=data+[polynomialCoefficient(n,poly)-nieburTau(n)]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
f=x^7
f.degree(x)

7

In [14]:
import pickle
vfile = open('/Users/barrybrent/8mar21no1.txt','r') # H4 series
# from Jupyter notebook "H4 from scratch"
rv=pickle.load(vfile)
vfile.close()
v=stripQuotationMarks(rv)
ln=len(v)
poly=v[0][1]
deg=poly.degree(x)
print([ln,v[0][0],v[-1][0],deg])


[498, 3, 500, 120]


In [15]:
import pickle
rfile = open('/Users/barrybrent/16mar21no13.txt','r') # H6 series
# from Jupyter notebook "H6 & H6 polynomials from scratch"
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
ln=len(s)
poly=s[0][1]
deg=poly.degree(x)
print([ln,s[0][0],s[-1][0],deg])

[398, 3, 400, 100]


In [10]:
import pickle
vfile = open('/Users/barrybrent/8mar21no1.txt','r') # H4 series
# from Jupyter notebook "H4 from scratch"
rv=pickle.load(vfile)
vfile.close()
v=stripQuotationMarks(rv)

import pickle
sfile = open('/Users/barrybrent/16mar21no13.txt','r') # H6 series
# from Jupyter notebook "H6 & H6 polynomials from scratch"
rs=pickle.load(sfile)
sfile.close()
s=stripQuotationMarks(rs)

import pickle
import time
wfile = open('/Users/barrybrent/19mar21no10.txt','w')
data=[]

for n in [0..0]:
    start = time.time()
    m=s[n][0]
    h6squared=s[n][1]^2
    h4cubed=v[n][1]^3
    dagger=D(((h4cubed-h6squared)/1728)).truncate(6)
    data=data+[[m,dagger]]
    finish = time.time()
    print([m,finish-start])
    print(dagger)
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

[3, 0.012415885925292969]
4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x


In [2]:
deltaDagger(5,3)

4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

In [16]:
import pickle
vfile = open('/Users/barrybrent/8mar21no1.txt','r') # H4 series
# from Jupyter notebook "H4 from scratch"
rv=pickle.load(vfile)
vfile.close()
v=stripQuotationMarks(rv)

import pickle
sfile = open('/Users/barrybrent/16mar21no13.txt','r') # H6 series
# from Jupyter notebook "H6 & H6 polynomials from scratch"
rs=pickle.load(sfile)
sfile.close()
s=stripQuotationMarks(rs)

import pickle
import time
wfile = open('/Users/barrybrent/19mar21no12.txt','w') # dagger series
data=[]

for n in [0..397]:
    start = time.time()
    m=s[n][0]
    h6squared=s[n][1]^2
    h4cubed=v[n][1]^3
    dagger=D(((h4cubed-h6squared)/1728)).truncate(100)
    data=data+[[m,dagger]]
    finish = time.time()
    print([m,finish-start])
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

[3, 0.012084007263183594]
[4, 0.01718282699584961]
[5, 0.030865907669067383]
[6, 0.013627052307128906]
[7, 0.032353878021240234]
[8, 0.03253889083862305]
[9, 0.03295493125915527]
[10, 0.03317999839782715]
[11, 0.03512692451477051]
[12, 0.032887935638427734]
[13, 0.03570199012756348]
[14, 0.03446006774902344]
[15, 0.03473401069641113]
[16, 0.03400111198425293]
[17, 0.035890817642211914]
[18, 0.0336458683013916]
[19, 0.03738903999328613]
[20, 0.03611612319946289]
[21, 0.03427600860595703]
[22, 0.0353548526763916]
[23, 0.036869049072265625]
[24, 0.03421187400817871]
[25, 0.036157846450805664]
[26, 0.037878990173339844]
[27, 0.03726506233215332]
[28, 0.03632807731628418]
[29, 0.03695392608642578]
[30, 0.03515291213989258]
[31, 0.03627300262451172]
[32, 0.038214921951293945]
[33, 0.039952993392944336]
[34, 0.03702402114868164]
[35, 0.03836202621459961]
[36, 0.0375521183013916]
[37, 0.03939509391784668]
[38, 0.038372039794921875]
[39, 0.037938833236694336]
[40, 0.038166046142578125]
[41, 0.0

[315, 0.0469820499420166]
[316, 0.04987597465515137]
[317, 0.04981803894042969]
[318, 0.04780697822570801]
[319, 0.047057151794433594]
[320, 0.046504974365234375]
[321, 0.04774308204650879]
[322, 0.048521995544433594]
[323, 0.050518035888671875]
[324, 0.04848194122314453]
[325, 0.048933982849121094]
[326, 0.04720115661621094]
[327, 0.04773712158203125]
[328, 0.04707908630371094]
[329, 0.049349069595336914]
[330, 0.04659104347229004]
[331, 0.04965090751647949]
[332, 0.049581050872802734]
[333, 0.04966306686401367]
[334, 0.0490419864654541]
[335, 0.046296119689941406]
[336, 0.044374942779541016]
[337, 0.048705101013183594]
[338, 0.0506439208984375]
[339, 0.04678201675415039]
[340, 0.04531693458557129]
[341, 0.047882080078125]
[342, 0.047808170318603516]
[343, 0.04944300651550293]
[344, 0.04825401306152344]
[345, 0.046792030334472656]
[346, 0.0470280647277832]
[347, 0.05100607872009277]
[348, 0.04861617088317871]
[349, 0.049139976501464844]
[350, 0.045269012451171875]
[351, 0.043759822845

In [3]:
import pickle
rfile = open('/Users/barrybrent/19mar21no12.txt','r') # dagger series
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    m = s[k][0]
    poly=D(s[k][1])
    data=data+[[k,poly.degree()]]
print(data) # correctness check: all of the series have the right degree.

[[0, 99], [1, 99], [2, 99], [3, 99], [4, 99], [5, 99], [6, 99], [7, 99], [8, 99], [9, 99], [10, 99], [11, 99], [12, 99], [13, 99], [14, 99], [15, 99], [16, 99], [17, 99], [18, 99], [19, 99], [20, 99], [21, 99], [22, 99], [23, 99], [24, 99], [25, 99], [26, 99], [27, 99], [28, 99], [29, 99], [30, 99], [31, 99], [32, 99], [33, 99], [34, 99], [35, 99], [36, 99], [37, 99], [38, 99], [39, 99], [40, 99], [41, 99], [42, 99], [43, 99], [44, 99], [45, 99], [46, 99], [47, 99], [48, 99], [49, 99], [50, 99], [51, 99], [52, 99], [53, 99], [54, 99], [55, 99], [56, 99], [57, 99], [58, 99], [59, 99], [60, 99], [61, 99], [62, 99], [63, 99], [64, 99], [65, 99], [66, 99], [67, 99], [68, 99], [69, 99], [70, 99], [71, 99], [72, 99], [73, 99], [74, 99], [75, 99], [76, 99], [77, 99], [78, 99], [79, 99], [80, 99], [81, 99], [82, 99], [83, 99], [84, 99], [85, 99], [86, 99], [87, 99], [88, 99], [89, 99], [90, 99], [91, 99], [92, 99], [93, 99], [94, 99], [95, 99], [96, 99], [97, 99], [98, 99], [99, 99], [100, 99]

In [2]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/19mar21no12.txt','r') # dagger series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/19mar21no15.txt','w') # dagger polynomials
for qpower in [0..99]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1] 
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower,rl]] 
    finish = time.time()
    print([qpower,finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 1.5345680713653564]
[1, 1.1433680057525635]
[2, 1.1281380653381348]
[3, 1.129424810409546]
[4, 1.1296570301055908]
[5, 1.1477470397949219]
[6, 1.1376008987426758]
[7, 1.1359190940856934]
[8, 1.1508300304412842]
[9, 1.115767002105713]
[10, 1.1650769710540771]
[11, 1.1811110973358154]
[12, 1.1648879051208496]
[13, 1.1701788902282715]
[14, 1.1603591442108154]
[15, 1.1751019954681396]
[16, 1.1785061359405518]
[17, 1.1733899116516113]
[18, 1.1281719207763672]
[19, 1.1877751350402832]
[20, 1.182305097579956]
[21, 1.1703369617462158]
[22, 1.1964049339294434]
[23, 1.1906781196594238]
[24, 1.1805400848388672]
[25, 1.189035177230835]
[26, 1.2012438774108887]
[27, 1.153630018234253]
[28, 1.1993520259857178]
[29, 1.232107162475586]
[30, 1.2443199157714844]
[31, 1.2360591888427734]
[32, 1.2511980533599854]
[33, 1.2269439697265625]
[34, 1.2525241374969482]
[35, 1.2735209465026855]
[36, 1.214920997619629]
[37, 1.2618727684020996]
[38, 1.2645108699798584]
[39, 1.263735055923462]
[40, 1.28194904327

In [3]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/19mar21no15.txt','rb')# dagger series  polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..6]:
    print("================================================")
    print("index: ",s[k][0])
    print("polynomial:")
    ff=D(s[k][1])
    f=ff.truncate(5)
    print(f)
    
             
             

index:  0
polynomial:
0
index:  1
polynomial:
1/36*x^3 + 1/108*x^2 + 1/18*x
index:  2
polynomial:
-11/18*x^4 + 65/27*x^3 + 10/9*x^2
index:  3
polynomial:
-2960/243*x^4 - 736/81*x^3
index:  4
polynomial:
24032/243*x^4
index:  5
polynomial:
0
index:  6
polynomial:
0
